# Settings all imports

In [4]:
import tqdm
import requests
import urllib.request

import numpy as np
import pandas as pd

from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from bs4.element import Comment
# from google.colab import drive

# Create a web driver based on operating system. Please change the value of the `OPERATING_SYSTEM` constant to run this notebook on your system

In [5]:
OPERATING_SYSTEM = "mac"

if OPERATING_SYSTEM == "windows":
    executable_path = "drivers/windows/chromedriver.exe"
elif OPERATING_SYSTEM == "mac":
    executable_path = "drivers/mac/chromedriver"
elif OPERATING_SYSTEM == "linux":
    executable_path = "chromedriver"

base_url = 'https://www.jobs.bg/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=executable_path, options=chrome_options)


"""url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')"""

'url = \'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging\'\nr = driver.get(url)\nhtml = driver.execute_script("return document.documentElement.outerHTML")\nsoup = BeautifulSoup(html, \'html.parser\')'

# Extracting the category names and ids so we can perform filters based on category

In [6]:
def extract_categories_dataframe():
    """"""
    url = "https://www.jobs.bg/index.php"
    r = driver.get(url)
    categories_button = driver.find_element_by_id("categoriesChip")
    categories_button.click()
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    categories_elem = soup.find('div',attrs={'id':'categoriesSelectSheet'})
    categories_elements = categories_elem.find_all("div", attrs={"class": "mdc-chip mdc-ripple-upgraded"})

    categories_info = []
    categories_info.append((56))

    for category_element in categories_elements:
        category_id = category_element['id'].split("_")[-1]
        category_id = int(category_id)
        categories_info.append(category_id)

    #categories_info = pd.DataFrame(categories_info, columns=["category_id"])

    return categories_info

In [15]:
categories_info = extract_categories_dataframe()
#categories_info.sort_values(by='category_id',inplace=True)
categories_info

[56,
 36,
 45,
 57,
 6,
 38,
 52,
 29,
 3,
 17,
 41,
 35,
 4,
 2,
 11,
 27,
 18,
 37,
 43,
 26,
 20,
 24,
 22,
 51,
 9,
 7,
 39,
 23,
 28,
 19,
 13,
 25,
 1,
 21]

In [56]:
def get_position_id(text_info_elem):
    """
    """
    class_id_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
    id_elem = text_info_elem.find('a',attrs={'class':class_id_elem})['href']
    position_id = id_elem.lstrip('job/')
    
    return position_id

def extract_job_info_as_array(base_url, text_info_elem, job_company_elem):
    """
    """
    position_id = get_position_id(text_info_elem)

    current_job_info = [position_id]
    
    return current_job_info

In [57]:
def extract_jobs(start_page, end_page):
    """"""
    jobs_list = []

    for category in tqdm.tqdm(categories_info):
        
        for page in tqdm.tqdm(range(start_page, end_page)):
    
            try:
                # This is the base url used when no filters by categories are applied
                url = "https://www.jobs.bg/front_job_search.php?frompage={}&add_sh=1&from_hp=1&term=#paging".format(page*15)
              
                if category:
                    url = "https://www.jobs.bg/front_job_search.php?frompage={}&add_sh=1&categories%5B0%5D={}&term=#paging".format(page*15, category)
              
                r = driver.get(url)
                html = driver.execute_script("return document.documentElement.outerHTML")
                soup = BeautifulSoup(html, 'html.parser')
                job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})
            except Exception as e:
                print(f"Page number: {page}, url: {url}")
                print(f"Unexpected error: {e.message}")
    
            for i in range(0, 45, 3):
                try:
                    text_info_elem = job_infos[i]
                    job_logo_elem = job_infos[i+1]
                    job_company_elem = job_infos[i+2]
    
                    current_job_info = extract_job_info_as_array(base_url, text_info_elem, job_company_elem)
                    current_job_info = (current_job_info, category)
                    jobs_list.append(current_job_info)
                except Exception as e:
                    print(f"Curent page: {page}, problem with the {i}, {i+1}, {i+2} tds")
                    print(f"Unexpected error: {e}")
    
        df_columns = ["position_id", "category"]
    
        jobs_df = pd.DataFrame(jobs_list, columns=df_columns)
    
        return jobs_df

In [58]:
jobs = extract_jobs(0,1)

  0%|          | 0/34 [00:00<?, ?it/s]


In [59]:
jobs

,position_id,category
0,[5707118],56
1,[5695491],56
2,[5673409],56
3,[5669897],56
4,[5667007],56
5,[5667011],56
6,[5666999],56
7,[5667461],56
8,[5695722],56
9,[5665209],56


In [14]:
for category in tqdm.tqdm(categories_info):
    try:
        url = 'https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D={}'.format(category)
        print(url)
    except:
        print('z')

100%|██████████| 34/34 [00:00<00:00, 20910.02it/s]

https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=56
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=36
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=45
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=57
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=6
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=38
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=52
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=29
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=3
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=17
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=41
https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1&categories%5B%5D=35
https://www.jobs.bg/front_job_search.php?a